# price -> volatility calculation

In [17]:
# load packages

import numpy as np
import matplotlib.pyplot as plt 



In [18]:
# generate price spot S_0

S = np.random.normal(2,np.sqrt(3),1000) # (mu, sigma, N)
np.var(S)

2.862965803747345

$ \frac{x}{1-x} = \frac{1}{1-x} - 1$

In [28]:
# BS Price calculation

def S_BS(sigma, r, S, t, T, K):
    
    from math import sqrt, log, exp
    import numpy as np
    from scipy.stats import norm
    
    d1 = (1/(sigma*np.sqrt(T-t)))*(np.log(S/K)+(r+(sigma**2)/2)*(T-t))
    d2 = d1-sigma*np.sqrt(T-t)
    print(d1)
    print(d2)
    
    return S*norm.cdf(d1)-(K*np.exp(-r*(T-t))*norm.cdf(d2))

In [29]:
# bi-section method

def imp_vol(S, r, t, T, K, N = 20): # N: iteration number
    
    import numpy as np
    
    n = N+1
    x = np.zeros(n)
    y = np.ones(n) # index : 0 -> N
    z = np.zeros(n)
    
    for i in range(1,n): # index : 0 -> N-1
        z[i] = 1/2 * (x[i-1] + y[i-1])
        sig = z[i] / (1 - z[i])
        if (S > S_BS(sig, r, S, t, T, K)):
            x[i] = z[i]
            y[i] = y[i-1]
        else:
            x[i] = x[i-1]
            y[i] = z[i]
            
    z = (x[N] + y[N]) / 2
    sigma = z / (1-z)
    
    return {'z': z, 'sigma': sigma }
        

In [31]:
a = imp_vol(S, 0,2, 10, 2)
plt.plot(a)

[ 1.23847278         nan  1.38344362  1.77876898  1.37043837  0.83425379
  1.42901038         nan  1.7592778   1.48127744  1.49820133         nan
  1.33397658  1.76642076  1.46097761  1.81250693  0.8852457   0.87008211
  1.4967346   1.52488925  1.21494193  0.61229172  1.24991474  1.4835947
  1.58179893  1.53656859  1.33700286         nan  1.7780623   1.35183216
  1.21750605  1.40186818         nan  1.20381521  1.59118908  1.6289332
  1.59463793  1.70345806  1.35799893  1.8497607   1.52830188  1.47515167
  1.61398139  1.59713372  1.30460405  1.5871093   1.64639073  1.4564892
  1.54248885         nan  1.49996328         nan  1.51848695  1.72921585
         nan  1.36430794  1.52812434  0.98893634  1.57364622         nan
  1.54999172  1.59333362  1.38838408  1.55622018  1.31593631  1.33658155
  1.60463156  1.2578022          nan  1.35400279  1.46110245  1.06535674
  1.57943484  1.08569501  1.43540587  1.69563669  1.58353741  1.94438175
  1.62038638  1.43247656  1.44076357  1.15176368  1.39

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log
  if __name__ == '__main__':
/usr/lib/python3/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/lib/python3/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/lib/python3/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater
  from ipykernel import kernelapp as app


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()